In [5]:
print('Importing packages...')
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

Importing packages...


In [6]:
print('Importing models from scikit learn module..')
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.cross_validation import KFold   
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
from sklearn import svm

Importing models from scikit learn module..


In [7]:
print('Reading data...')
dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')

Reading data...


In [8]:
 dfTrain.apply(lambda x: sum(x.isnull()),axis=0)  # checking number of null values in each column

LOAN_ID           0
AMOUNT            0
DUE_MORTGAGE    318
VALUE            67
REASON          157
OCC             173
TJOB            309
DCL             337
CLT             187
CL_COUNT        130
RATIO           751
CONVICTED         0
VAR_1             0
VAR_2             0
VAR_3             0
DEFAULTER         0
dtype: int64

In [9]:
null_columns=dfTrain.columns[dfTrain.isnull().any()]   # finding all rows which do not have null value in ratio
df_ratio_notnull=dfTrain[dfTrain["RATIO"].notnull()][null_columns]

In [10]:
df_ratio_notnull.head()

,DUE_MORTGAGE,VALUE,REASON,OCC,TJOB,DCL,CLT,CL_COUNT,RATIO
1,81747.0,109840.0,1.0,0.0,NaN,0.0,174.786162,24.0,40.067527
2,94262.0,128414.0,1.0,3.0,4.0,NaN,242.736126,42.0,39.231892
4,NaN,35523.0,0.0,0.0,21.0,0.0,84.111988,15.0,29.865634
5,65742.0,86608.0,0.0,0.0,NaN,0.0,289.581891,20.0,28.285563
6,75253.0,106166.0,1.0,0.0,NaN,0.0,176.339376,24.0,38.266594


In [109]:
cols = ['DUE_MORTGAGE','VALUE','TJOB','DCL','CLT','CL_COUNT']   # imputing the data to make up for missing data
for col in cols: 
    print('Imputation with Median: %s' % (col))
    df_ratio_notnull[col].fillna(df_ratio_notnull[col].median(), inplace=True)
    dfTest[col].fillna(dfTest[col].median(), inplace=True)

    cols=['OCC']
for col in cols:
    print('Imputation with Zero: %s' % (col))
    df_ratio_notnull[col].fillna(0, inplace=True)
    dfTest[col].fillna(0, inplace=True)
    
    
    cols=['REASON']
for col in cols:
    print('Imputation with One: %s' % (col))
    df_ratio_notnull[col].fillna(1, inplace=True)
    dfTest[col].fillna(1, inplace=True)

Imputation with Median: DUE_MORTGAGE
Imputation with Median: VALUE
Imputation with Median: TJOB
Imputation with Median: DCL
Imputation with Median: CLT
Imputation with Median: CL_COUNT
Imputation with Zero: OCC
Imputation with One: REASON


In [110]:
train_target = pd.DataFrame(df_ratio_notnull['RATIO'])   #training a model to learn the values of RATIO
train_cols=['DUE_MORTGAGE','VALUE','REASON','OCC','TJOB','DCL','CLT','CL_COUNT']
finalTrain=df_ratio_notnull[train_cols];
reg=LinearRegression()
reg.fit(finalTrain,train_target)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [111]:
dfTrain.head()

,LOAN_ID,AMOUNT,DUE_MORTGAGE,VALUE,REASON,OCC,TJOB,DCL,CLT,CL_COUNT,RATIO,CONVICTED,VAR_1,VAR_2,VAR_3,DEFAULTER
0,3682648,15100,71168.0,97480.0,1.0,0.0,9.0,0.0,119.166667,19.0,NaN,0,0,1,2.0095,1
1,3682649,21100,81747.0,109840.0,1.0,0.0,NaN,0.0,174.786162,24.0,40.067527,0,0,1,1.2724,0
2,3682650,15900,94262.0,128414.0,1.0,3.0,4.0,NaN,242.736126,42.0,39.231892,1,0,2,1.4725,0
3,3682651,12900,110899.0,124480.0,1.0,2.0,7.0,0.0,204.900000,40.0,NaN,0,1,2,1.5701,1
4,3682652,24600,NaN,35523.0,0.0,0.0,21.0,0.0,84.111988,15.0,29.865634,0,0,1,1.5013,0


In [112]:
dfTrain.describe()

,LOAN_ID,AMOUNT,DUE_MORTGAGE,VALUE,REASON,OCC,TJOB,DCL,CLT,CL_COUNT,RATIO,CONVICTED,VAR_1,VAR_2,VAR_3,DEFAULTER
count,3.576000e+03,3576.000000,3258.000000,3509.000000,3419.000000,3403.000000,3267.000000,3239.000000,3389.000000,3446.000000,2825.000000,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000
mean,3.684436e+06,18593.596197,74028.507532,101706.385571,0.690553,1.666765,8.895301,0.428836,181.339737,21.161056,33.650308,0.171141,0.218400,1.095358,1.658922,0.197707
std,1.032447e+03,10901.127719,44716.291776,55426.276328,0.462333,1.747940,7.591961,1.108302,83.487218,10.150044,7.873496,0.376685,0.768329,1.719366,0.662931,0.398326
min,3.682648e+06,1200.000000,1962.000000,7980.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.730295,0.000000,0.000000,0.000000,0.045274,0.000000
25%,3.683542e+06,11200.000000,46546.750000,66395.000000,0.000000,0.000000,3.000000,0.000000,117.437908,14.000000,29.161698,0.000000,0.000000,0.000000,1.184400,0.000000
50%,3.684436e+06,16400.000000,65849.500000,89569.000000,1.000000,1.000000,7.000000,0.000000,174.500000,20.000000,34.644236,0.000000,0.000000,0.000000,1.702700,0.000000
75%,3.685329e+06,23200.000000,91288.000000,120159.000000,1.000000,4.000000,13.000000,0.000000,233.333333,26.000000,38.958919,0.000000,0.000000,2.000000,2.188450,0.000000
max,3.686223e+06,89900.000000,399449.000000,849980.000000,1.000000,5.000000,41.000000,15.000000,640.058172,71.000000,143.959605,1.000000,10.000000,17.000000,2.779000,1.000000


In [113]:
dfTrain['REASON'].value_counts()

1.0    2361
0.0    1058
Name: REASON, dtype: int64

In [170]:
dfTrain['DEFAULTER'].value_counts()

0    2869
1     707
Name: DEFAULTER, dtype: int64

In [114]:
dfTrain['OCC'].value_counts()

0.0    1433
4.0     756
1.0     574
3.0     470
5.0     108
2.0      62
Name: OCC, dtype: int64

In [13]:
cols = ['AMOUNT','DUE_MORTGAGE','VALUE','TJOB','DCL','CLT','CL_COUNT']
for col in cols:
    print('Imputation with Median: %s' % (col))
    dfTrain[col].fillna(dfTrain[col].median(), inplace=True)
    dfTest[col].fillna(dfTest[col].median(), inplace=True)

    cols=['OCC']
for col in cols:
    print('Imputation with Zero: %s' % (col))
    dfTrain[col].fillna(0, inplace=True)
    dfTest[col].fillna(0, inplace=True)
    
    
    cols=['REASON']
for col in cols:
    print('Imputation with One: %s' % (col))
    dfTrain[col].fillna(1, inplace=True)
    dfTest[col].fillna(1, inplace=True)

Imputation with Median: AMOUNT
Imputation with Median: DUE_MORTGAGE
Imputation with Median: VALUE
Imputation with Median: TJOB
Imputation with Median: DCL
Imputation with Median: CLT
Imputation with Median: CL_COUNT
Imputation with Zero: OCC
Imputation with One: REASON


In [117]:
pred=reg.predict(dfTrain[train_cols])

In [118]:
dfTrain['RATIO']=pred # putting the predicted valus of ratio in the dataframe

In [119]:
dfTrain['log_amount'] = np.log(dfTrain['AMOUNT'])  # adding extra columns by performing log transformation
dfTrain['log_Due_Mortgage'] = np.log(dfTrain['DUE_MORTGAGE'])
dfTrain['log_value'] = np.log(dfTrain['VALUE'])

In [120]:
train_target = pd.DataFrame(dfTrain['DEFAULTER'])

In [121]:
train_cols=['log_amount','log_Due_Mortgage','log_value','REASON','OCC','TJOB','DCL','CLT','CL_COUNT','RATIO','CONVICTED','VAR_1','VAR_2','VAR_3']

In [122]:
finalTrain=dfTrain[train_cols];

In [123]:
X_train, X_test, y_train, y_test = train_test_split(np.array(finalTrain), np.array(train_target), test_size=0.30,random_state=59)

In [124]:
clf = LogisticRegression()  # logistic regression

In [125]:
clf.fit(X_train, y_train)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [126]:
y_pred = clf.predict(X_test)
print('Accuracy of logistic regression classifier on test set:')
print(clf.score(X_test, y_test))

Accuracy of logistic regression classifier on test set:
0.835973904939


In [127]:
from sklearn.preprocessing import StandardScaler  # scaling the data so that neural network works better
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [128]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [129]:
from sklearn.neural_network import MLPClassifier

In [130]:
mlp = MLPClassifier(hidden_layer_sizes=(100,100,50,30,10),solver='adam',max_iter=1000,verbose=True)  # neural network

In [131]:
mlp.fit(X_train,y_train)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.55360149
Iteration 2, loss = 0.43330239
Iteration 3, loss = 0.39032165
Iteration 4, loss = 0.37576040
Iteration 5, loss = 0.36309049
Iteration 6, loss = 0.35084660
Iteration 7, loss = 0.34099132
Iteration 8, loss = 0.32855016
Iteration 9, loss = 0.31372944
Iteration 10, loss = 0.30128113
Iteration 11, loss = 0.28532252
Iteration 12, loss = 0.27213172
Iteration 13, loss = 0.25957506
Iteration 14, loss = 0.24188795
Iteration 15, loss = 0.22840395
Iteration 16, loss = 0.21435258
Iteration 17, loss = 0.20245274
Iteration 18, loss = 0.19014191
Iteration 19, loss = 0.17378612
Iteration 20, loss = 0.17011668
Iteration 21, loss = 0.14927321
Iteration 22, loss = 0.13402509
Iteration 23, loss = 0.12390630
Iteration 24, loss = 0.11099031
Iteration 25, loss = 0.10250533
Iteration 26, loss = 0.08828110
Iteration 27, loss = 0.07892033
Iteration 28, loss = 0.07795833
Iteration 29, loss = 0.06574483
Iteration 30, loss = 0.05823559
Iteration 31, loss = 0.06106706
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 50, 30, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [132]:
predictions=mlp.predict(X_test)

In [133]:
print('Accuracy of neural network classifier on test set:')
print(mlp.score(X_test, y_test))

Accuracy of neural network classifier on test set:
0.880708294501


In [134]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,predictions)

0.77295907342581793

In [135]:
dfTrain['annual_income']=dfTrain['AMOUNT']/dfTrain['RATIO'] # adding an extra column of annual_income

In [136]:
dfTrain.describe()

,LOAN_ID,AMOUNT,DUE_MORTGAGE,VALUE,REASON,OCC,TJOB,DCL,CLT,CL_COUNT,RATIO,CONVICTED,VAR_1,VAR_2,VAR_3,DEFAULTER,log_amount,log_Due_Mortgage,log_value,annual_income
count,3.576000e+03,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000,3576.000000
mean,3.684436e+06,18593.596197,73301.179681,101478.979298,0.704139,1.586130,8.731530,0.388423,180.982066,21.118848,33.859044,0.171141,0.218400,1.095358,1.658922,0.197707,9.675788,11.037601,11.405230,605.109393
std,1.032447e+03,10901.127719,42744.663989,54929.106774,0.456493,1.742233,7.275966,1.062189,81.288647,9.966159,7.009511,0.376685,0.768329,1.719366,0.662931,0.398326,0.570761,0.615122,0.496601,821.210259
min,3.682648e+06,1200.000000,1962.000000,7980.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.730295,0.000000,0.000000,0.000000,0.045274,0.000000,7.090077,7.581720,8.984694,34.637797
25%,3.683542e+06,11200.000000,48522.500000,66723.000000,0.000000,0.000000,3.000000,0.000000,119.431208,14.000000,30.773159,0.000000,0.000000,0.000000,1.184400,0.000000,9.323669,10.789783,11.108305,333.911294
50%,3.684436e+06,16400.000000,65849.500000,89569.000000,1.000000,1.000000,7.000000,0.000000,174.500000,20.000000,34.644236,0.000000,0.000000,0.000000,1.702700,0.000000,9.705037,11.095127,11.402765,493.588607
75%,3.685329e+06,23200.000000,88043.000000,119132.000000,1.000000,3.000000,12.000000,0.000000,228.722132,26.000000,37.944475,0.000000,0.000000,2.000000,2.188450,0.000000,10.051908,11.385581,11.687987,693.865496
max,3.686223e+06,89900.000000,399449.000000,849980.000000,1.000000,5.000000,41.000000,15.000000,640.058172,71.000000,143.959605,1.000000,10.000000,17.000000,2.779000,1.000000,11.406453,12.897841,13.652968,33014.292447


In [137]:
train_cols=['log_amount','log_Due_Mortgage','log_value','REASON','OCC','TJOB','DCL','CLT','CL_COUNT','RATIO','CONVICTED','annual_income','VAR_1','VAR_2','VAR_3']

In [138]:
finalTrain=dfTrain[train_cols];

In [139]:
X_train, X_test, y_train, y_test = train_test_split(np.array(finalTrain), np.array(train_target), test_size=0.30,random_state=59)

In [140]:
clf=LogisticRegression()  #logistic Rgression __ again

In [141]:
clf.fit(X_train,y_train)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [142]:
y_pred = clf.predict(X_test)
print('Accuracy of logistic regression classifier on test set:')
print(clf.score(X_test, y_test))

Accuracy of logistic regression classifier on test set:
0.838769804287


In [143]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,y_pred)

0.63762749038419997

In [144]:
mlp_1= MLPClassifier(hidden_layer_sizes=(30,30,30),solver='adam')  # another neural network model with different number of hidden layers 

In [145]:
mlp_1.fit(X_train,y_train)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(30, 30, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [146]:
y_pred_1 = mlp_1.predict(X_test)
print('Accuracy of neural network classifier on test set:')
print(mlp_1.score(X_test, y_test))

Accuracy of neural network classifier on test set:
0.789375582479


In [147]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,y_pred_1)

0.5738093694628118

In [148]:
from sklearn import svm  # svm classifier

In [149]:
svm_clf = svm.SVC(kernel='linear')

In [150]:
svm_clf.fit(X_train,y_train)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [151]:
pred=svm_clf.predict(X_test)

In [152]:
print('Accuracy of svm classifier on test set:')
print(svm_clf.score(X_test, y_test))

Accuracy of svm classifier on test set:
0.834109972041


In [153]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,pred)

0.61912787933791436

In [154]:
dfTest.head()

,TEST_ID,LOAN_ID,AMOUNT,DUE_MORTGAGE,VALUE,REASON,OCC,TJOB,DCL,CLT,CL_COUNT,RATIO,CONVICTED,VAR_1,VAR_2,VAR_3
0,25667,3686224,12300,67373.0,88453.0,1.0,0.0,2.0,0.0,235.775997,26.0,29.107299,0,0,3,2.1966
1,25668,3686225,45100,63955.5,76653.0,0.0,1.0,11.0,0.0,298.300000,13.0,35.068381,0,0,0,2.4432
2,25669,3686226,28400,129927.0,178898.0,1.0,3.0,2.0,0.0,114.942639,39.0,33.450162,0,0,0,1.0867
3,25670,3686227,8400,101049.0,115195.0,0.0,1.0,23.0,0.0,131.256329,12.0,36.515999,0,0,0,1.5343
4,25671,3686228,20000,16131.0,70572.0,1.0,1.0,7.0,0.0,243.365403,22.0,39.456251,0,0,0,2.3169


In [160]:
dfTest['log_amount'] = np.log(dfTest['AMOUNT'])  # preparing the test data
dfTest['log_Due_Mortgage'] = np.log(dfTest['DUE_MORTGAGE'])
dfTest['log_value'] = np.log(dfTest['VALUE'])

In [161]:
test_cols=['log_amount','log_Due_Mortgage','log_value','REASON','OCC','TJOB','DCL','CLT','CL_COUNT','RATIO','CONVICTED','VAR_1','VAR_2','VAR_3']

In [162]:
final_test=dfTest[test_cols]

In [164]:
predictions=mlp.predict(final_test) # using the neural network classifier for predictions

In [168]:
import numpy as np
a = predictions
print(a)
unique_elements, counts_elements = np.unique(a, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

[0 0 0 ..., 0 0 1]
Frequency of unique values of the said array:
[[   0    1]
 [2210  174]]


In [174]:
dfTest_2=dfTest.drop(['TEST_ID','VALUE','DUE_MORTGAGE','AMOUNT','log_amount','log_Due_Mortgage','log_value','REASON','OCC','TJOB','DCL','CLT','CL_COUNT','RATIO','CONVICTED','VAR_1','VAR_2','VAR_3'],axis=1)

In [175]:
dfTest_2.head()

,LOAN_ID
0,3686224
1,3686225
2,3686226
3,3686227
4,3686228


In [176]:
dfTest_2['DEFAULTER']=predictions

In [178]:
dfTest_2.describe()

,LOAN_ID,DEFAULTER
count,2.384000e+03,2384.000000
mean,3.687416e+06,0.072987
std,6.883458e+02,0.260169
min,3.686224e+06,0.000000
25%,3.686820e+06,0.000000
50%,3.687416e+06,0.000000
75%,3.688011e+06,0.000000
max,3.688607e+06,1.000000


In [179]:
writer=pd.ExcelWriter('predictions.xlsx',engine='xlsxwriter')

In [180]:
dfTest_2.to_excel(writer,'Sheet1')

In [181]:
writer.save()